# Configure Sugar


In [ ]:
#| default_exp config

In [ ]:
#| hide
from nbdev.showdoc import *


# load env
from dotenv import load_dotenv
load_dotenv()

False

In [ ]:
#| export

import os
from dataclasses import dataclass, make_dataclass
from sugar.helpers import normalize_address
from typing import Literal, List, Optional
from web3 import AsyncWeb3, AsyncHTTPProvider

## Default velo settings

In [ ]:
#| export

DEFAULT_CONNECTORS_VELO = "0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db,0x4200000000000000000000000000000000000042,0x4200000000000000000000000000000000000006,0x9bcef72be871e61ed4fbbc7630889bee758eb81d,0x2e3d870790dc77a83dd1d18184acc7439a53f475,0x8c6f28f2f1a3c87f0f938b96d27520d9751ec8d9,0x1f32b1c2345538c0c6f582fcb022739c4a194ebb,0xbfd291da8a403daaf7e5e9dc1ec0aceacd4848b9,0xc3864f98f2a61a7caeb95b039d031b4e2f55e0e9,0x9485aca5bbbe1667ad97c7fe7c4531a624c8b1ed,0xDA10009cBd5D07dd0CeCc66161FC93D7c9000da1,0x73cb180bf0521828d8849bc8cf2b920918e23032,0x6806411765af15bddd26f8f544a34cc40cb9838b,0x6c2f7b6110a37b3b0fbdd811876be368df02e8b0,0xc5b001dc33727f8f26880b184090d3e252470d45,0x6c84a8f1c29108f47a79964b5fe888d4f4d0de40,0xc40f949f8a4e094d1b49a23ea9241d289b7b2819,0x94b008aa00579c1307b0ef2c499ad98a8ce58e58,0x0b2c639c533813f4aa9d7837caf62653d097ff85" 
DEFAULT_CONNECTORS_AERO = "0x833589fcd6edb6e08f4c7c32d4f71b54bda02913,0x940181a94A35A4569E4529A3CDfB74e38FD98631,0x50c5725949A6F0c72E6C4a641F24049A917DB0Cb,0x4621b7a9c75199271f773ebd9a499dbd165c3191,0x4200000000000000000000000000000000000006,0xb79dd08ea68a908a97220c76d19a6aa9cbde4376,0xf7a0dd3317535ec4f4d29adf9d620b3d8d5d5069,0xcfa3ef56d303ae4faaba0592388f19d7c3399fb4,0xcb327b99ff831bf8223cced12b1338ff3aa322ff,0x2ae3f1ec7f1f5012cfeab0185bfc7aa3cf0dec22,0xc1cba3fcea344f92d9239c08c0568f6f2f0ee452,0x60a3e35cc302bfa44cb288bc5a4f316fdb1adb42,0xd9aaec86b65d86f6a7b5b1b0c42ffa531710b6ca,0xcbB7C0000aB88B473b1f5aFd9ef808440eed33Bf"

base_default_settings = {
  "price_batch_size": int(os.getenv("SUGAR_PRICE_BATCH_SIZE","40")),
  "pagination_limit": int(os.getenv("SUGAR_PAGINATION_LIMIT","2000"))
}

velo_default_settings = { **{
  "rpc_uri": os.getenv("SUGAR_RPC_URI", "https://optimism-mainnet.wallet.coinbase.com"),
  "sugar_contract_addr": os.getenv("SUGAR_CONTRACT_ADDR", "0x3B919747B46B13CFfd9f16629cFf951C0b7ea1e2"),
  "price_oracle_contract_addr": os.getenv("SUGAR_PRICE_ORACLE_ADDR", "0x59114D308C6DE4A84F5F8cD80485a5481047b99f"),
  "token_addr": normalize_address(os.getenv("SUGAR_TOKEN_ADDR", "0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db")),
  "stable_token_addr": normalize_address(os.getenv("SUGAR_STABLE_TOKEN_ADDR", "0x7F5c764cBc14f9669B88837ca1490cCa17c31607")),
  "connector_tokens_addrs": list(map(lambda a: normalize_address(a), os.getenv("SUGAR_CONNECTOR_TOKENS_ADDRS", DEFAULT_CONNECTORS_VELO).split(","))),
  "protocol_name": os.getenv("SUGAR_PROTOCOL_NAME","velo"),
}, **base_default_settings }

aero_default_settings = { **{
  "rpc_uri": os.getenv("SUGAR_RPC_URI", "https://mainnet.base.org"),
  "sugar_contract_addr": os.getenv("SUGAR_CONTRACT_ADDR", "0x92294D631E995f1dd9CeE4097426e6a71aB87Bcf"),
  "price_oracle_contract_addr": os.getenv("SUGAR_PRICE_ORACLE_ADDR", "0x3B06c787711ecb5624cE65AC8F26cde10831eb0C"),
  "token_addr": normalize_address(os.getenv("SUGAR_TOKEN_ADDR", "0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db")),
  "stable_token_addr": normalize_address(os.getenv("SUGAR_STABLE_TOKEN_ADDR", "0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA")),
  "connector_tokens_addrs": list(map(lambda a: normalize_address(a), os.getenv("SUGAR_CONNECTOR_TOKENS_ADDRS", DEFAULT_CONNECTORS_AERO).split(","))),
  "protocol_name": os.getenv("SUGAR_PROTOCOL_NAME","aero"),
}, **base_default_settings }         


In [ ]:
#| export

def _make_config(settings, **kwargs) -> 'SugarConfig':
    d = { **settings, **kwargs }
    d = { **d, "web3": AsyncWeb3(AsyncHTTPProvider(d["rpc_uri"]))}
    return make_dataclass(
      SugarConfig.__name__, ((k, type(v)) for k, v in d.items())
    )(**d)

@dataclass
class SugarConfig:
    _instance = None

    rpc_uri: str
    sugar_contract_addr: str
    price_oracle_contract_addr: str
    token_addr: str
    stable_token_addr: str
    connector_tokens_addrs: List[str]
    price_batch_size: int
    protocol_name: Literal["velo", "aero"]
    pagination_limit: int
    web3: AsyncWeb3
    
    @staticmethod
    def get_config() -> 'SugarConfig':
        if SugarConfig._instance is None: return SugarConfig.velo()
        return SugarConfig._instance

    @staticmethod
    def velo(**kwargs) -> 'SugarConfig': 
        SugarConfig._instance = _make_config(velo_default_settings, **kwargs)
        return SugarConfig._instance
    
    @staticmethod
    def aero(**kwargs) -> 'SugarConfig':
        SugarConfig._instance = _make_config(aero_default_settings, **kwargs)
        return SugarConfig._instance
        

    def __str__(self):
        # go over all attributes of self
        lines = ["🍭 Sugar Config:","----------------"]
        attributes = [attr for attr in dir(self) if not attr.startswith('_')]
        for attr in attributes:
            # Skip methods
            if callable(getattr(self, attr)): continue
            value = getattr(self, attr)
            if isinstance(value, list):
                value = "\n" + "\n".join([f"{' ' * 4 }- {v}" for v in value])
            lines.append(f"{attr}: {value}")
        return "\n".join(lines)
        
        
    def __repr__(self): return str(self)

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()